**Importing Required Packages**

In [9]:
import torch
import torchvision
from torch import nn, optim

import torch.nn.functional as F
from torchsummary import summary

**Configuring Model**

In [10]:
# Model Configs
batch_size = 64
learning_rate = 0.01
cross_entropy = nn.CrossEntropyLoss()

**Load MNIST Data**

In [11]:
# Data Loader
transform = torchvision.transforms.ToTensor()
train_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=True, download=True, transform=transform
    ), batch_size=batch_size
)
val_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=False, download=True, transform=transform
    ), batch_size=batch_size
)

**Define Validation Function**

Function to calculate accuracy of the given validation data

In [12]:
# Validation function
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        labels = labels.cuda()
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total

**Define Training Function**

Function to train the model on the training data

In [13]:
# Training Function
def train(model,epochs=5) :
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)    
    for n in range(epochs)  :
        for i , (images , labels) in enumerate(train_data) :
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            prediction = model(images)
            loss = cross_entropy(prediction, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_data))
        print("Epoch:", n+1, "Loss: ", float(loss.data), "Accuracy:", accuracy)

**Define Model**

In [14]:
# Model
class CNN(nn.Module) :
    def __init__(self):
        super(CNN,self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3)
        self.dense_1 = nn.Linear(in_features=9216,out_features=2048)
        self.dense_2 = nn.Linear(in_features=2048,out_features=10)

        self.tanh = nn.Tanh()
    def forward(self,x) :
        x = self.tanh(self.conv_1(x))
        x = self.tanh(self.conv_2(x))
        x = x.view(x.shape[0],-1)
        x = self.tanh(self.dense_1(x))
        x = self.dense_2(x)
        # output = self.tanh(x)
        output = F.log_softmax(x, dim=1)

        return output

**Create Model Instance**

In [15]:
# Model
model = CNN().cuda()

**Model Summary**

In [16]:
# Summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              Tanh-2            [-1, 8, 26, 26]               0
            Conv2d-3           [-1, 16, 24, 24]           1,168
              Tanh-4           [-1, 16, 24, 24]               0
            Linear-5                 [-1, 2048]      18,876,416
              Tanh-6                 [-1, 2048]               0
            Linear-7                   [-1, 10]          20,490
Total params: 18,898,154
Trainable params: 18,898,154
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 72.09
Estimated Total Size (MB): 72.35
----------------------------------------------------------------


**Train the model**

In [17]:
# Train for 30 Epochs
train(model,epochs=30)

Epoch: 1 Loss:  0.11636315286159515 Accuracy: 80.95999908447266
Epoch: 2 Loss:  0.5661401152610779 Accuracy: 83.7699966430664
Epoch: 3 Loss:  0.2669124901294708 Accuracy: 78.75999450683594
Epoch: 4 Loss:  0.023944273591041565 Accuracy: 76.33999633789062
Epoch: 5 Loss:  0.19102716445922852 Accuracy: 73.7699966430664
Epoch: 6 Loss:  0.3458019196987152 Accuracy: 80.75
Epoch: 7 Loss:  0.8541619181632996 Accuracy: 73.97999572753906
Epoch: 8 Loss:  0.6575016975402832 Accuracy: 87.32999420166016
Epoch: 9 Loss:  0.9873948097229004 Accuracy: 79.68999481201172
Epoch: 10 Loss:  1.2695802450180054 Accuracy: 54.78999710083008
Epoch: 11 Loss:  0.5420165657997131 Accuracy: 80.43999481201172
Epoch: 12 Loss:  0.406747967004776 Accuracy: 75.54000091552734
Epoch: 13 Loss:  0.20363746583461761 Accuracy: 77.48999786376953
Epoch: 14 Loss:  0.1979249268770218 Accuracy: 90.23999786376953
Epoch: 15 Loss:  0.1711551994085312 Accuracy: 83.43999481201172
Epoch: 16 Loss:  0.367438942193985 Accuracy: 78.12999725341

We can see that the model is able to try and reduce the loss function with varying validation accuracies.